### ANLY-580 Week 7 Homework
### Yi Li

In [1]:
## Assume one document per line
## The 1st-3rd lines come from one real article about General Motors
## The 4th-6th lines come from one real article about DigitalOcean
## The 7th-9th lines come from one real article about North Korea

# load my corpus
texts = open('mycorpus.txt')
for line in texts:
    print(line)

General Motors Company is up 1.65%, to $45.60 Friday, hitting new 52-week and all-time highs on the day. Ford Motor Company , on the other hand, is down 0.9% to about $12. Why the disparity?

The research report really pits General Motors against Tesla , making the case that General Motors is a lot further along in the "electrification and autonomy" of vehicles than many people give it credit for.

Maybe General Motors has more of an edge on Tesla than many investors currently realize. Whether that brings Tesla stock back to earth is still an unanswered question. It is a cult stock after all, he concluded.

While Amazon and Microsoft battle in the cloud wars, this startup quietly built a $175 million business by picking up their slack (AMZN, MSFT, GOOG, GOOGL)

DigitalOcean is a cloud computing startup based in New York, competing with Amazon Web Services and Microsoft Azure by focusing on smaller developers and startups.

"We're approaching the cloud market from a totally different an

In [2]:
from gensim import corpora
# collect statistics about all tokens
dictionary = corpora.Dictionary(line.lower().replace(',','').replace('.','').replace('?','').split() 
                                for line in open('mycorpus.txt'))
print(dictionary)

Dictionary(167 unique tokens: ['general', 'motors', 'company', 'is', 'up']...)


In [3]:
class MyCorpus(object):
    def __iter__(self):
        for line in open('mycorpus.txt'):
            # assume there's one document per line, tokens separated by whitespace
            yield dictionary.doc2bow(line.lower().replace(',','').replace('.','').replace('?','').split())

In [4]:
corpus_memory_friendly = MyCorpus()  # doesn't load the corpus into memory!
print(corpus_memory_friendly)

In [5]:
# remove stop words and words that appear only once
from nltk.corpus import stopwords
stoplist = set(stopwords.words('english'))
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist
            if stopword in dictionary.token2id]

from six import iteritems
once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1]
dictionary.filter_tokens(stop_ids + once_ids)  # remove stop words and words that appear only once
dictionary.compactify()  # remove gaps in id sequence after words that were removed
#print(dictionary)
print([[(dictionary[posi], freq) for posi, freq in vector]
     for vector in corpus_memory_friendly])

[[('general', 1), ('motors', 1), ('new', 1)], [('general', 2), ('motors', 2), ('really', 1), ('tesla', 1), ('many', 1)], [('general', 1), ('motors', 1), ('tesla', 2), ('many', 1)], [('amazon', 1), ('microsoft', 1), ('cloud', 1), ('startup', 1)], [('new', 1), ('amazon', 1), ('microsoft', 1), ('cloud', 1), ('startup', 1), ('digitalocean', 1), ('smaller', 1)], [('really', 1), ('cloud', 1), ('digitalocean', 1), ('smaller', 1), ('says', 1)], [('north', 1), ('korea', 1), ('nuclear', 1), ('tests', 1)], [('says', 1), ('north', 1)], [('north', 1), ('korea', 1), ('nuclear', 1), ('tests', 1)]]


In [6]:
corpus = [vector for vector in corpus_memory_friendly]

from gensim import corpora, models, similarities

## TF-IDF
tfidf = models.TfidfModel(corpus, normalize=True)
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
    print([(dictionary[posi], freq) for posi, freq in doc])

[('general', 0.5080429008916749), ('motors', 0.5080429008916749), ('new', 0.695546419520037)]
[('general', 0.5418669471286024), ('motors', 0.5418669471286024), ('really', 0.37092695741841886), ('tesla', 0.37092695741841886), ('many', 0.37092695741841886)]
[('general', 0.2965418751259446), ('motors', 0.2965418751259446), ('tesla', 0.8119733161101181), ('many', 0.40598665805505907)]
[('amazon', 0.5319813250092166), ('microsoft', 0.5319813250092166), ('cloud', 0.38857124124710346), ('startup', 0.5319813250092166)]
[('new', 0.3912248890996717), ('amazon', 0.3912248890996717), ('microsoft', 0.3912248890996717), ('cloud', 0.2857595438365925), ('startup', 0.3912248890996717), ('digitalocean', 0.3912248890996717), ('smaller', 0.3912248890996717)]
[('really', 0.4696586869775006), ('cloud', 0.34304937106235917), ('digitalocean', 0.4696586869775006), ('smaller', 0.4696586869775006), ('says', 0.4696586869775006)]
[('north', 0.38857124124710346), ('korea', 0.5319813250092166), ('nuclear', 0.5319813

In [13]:
## use cosine distance to calculate the similarity between two documents by TF-IDF weights
def get_cosine(vec1, vec2):
    cosine = 0
    for word1, weight1 in vec1:
        for word2, weight2 in vec2:
            if word1 == word2:
                cosine += weight1*weight2
    return cosine
  
print('Similarities between 1st-3rd documents:')
print('1st and 2nd:',get_cosine(corpus_tfidf[0], corpus_tfidf[1]))        
print('2nd and 3rd:',get_cosine(corpus_tfidf[1], corpus_tfidf[2]))
print('1st and 3rd:',get_cosine(corpus_tfidf[0], corpus_tfidf[1]))

print('Similarities between 4th-6th documents:')
print('4th and 5th:',get_cosine(corpus_tfidf[3], corpus_tfidf[4]))        
print('5th and 6th:',get_cosine(corpus_tfidf[4], corpus_tfidf[5]))
print('4th and 6th:',get_cosine(corpus_tfidf[3], corpus_tfidf[5]))

print('Similarities between 7th-9th documents:')
print('7th and 8th:',get_cosine(corpus_tfidf[6], corpus_tfidf[7]))        
print('8th and 9th:',get_cosine(corpus_tfidf[7], corpus_tfidf[8]))
print('7th and 9th:',get_cosine(corpus_tfidf[6], corpus_tfidf[8]))

print('Similarities between documents from different real articles:')
print('1st and 4th:',get_cosine(corpus_tfidf[1], corpus_tfidf[4]))        
print('4th and 8th:',get_cosine(corpus_tfidf[4], corpus_tfidf[8]))
print('1st and 8th:',get_cosine(corpus_tfidf[1], corpus_tfidf[8]))

Similarities between 1st-3rd documents:
1st and 2nd: 0.550583311433062
2nd and 3rd: 0.7731466686150789
1st and 3rd: 0.550583311433062
Similarities between 4th-6th documents:
4th and 5th: 0.7354109452862723
5th and 6th: 0.46551396724314986
4th and 6th: 0.13329911992273907
Similarities between 7th-9th documents:
7th and 8th: 0.22919259272018386
8th and 9th: 0.22919259272018386
7th and 9th: 0.9999999999999999
Similarities between documents from different real articles:
1st and 4th: 0
4th and 8th: 0
1st and 8th: 0


I used the TF-IDF weights to calculate the cosine similarities between different documents. The results show that 
* this method is good at identifying the documents from different real articles (becuase the similarity socre is 0), 
* but this method is no quite good at identifying the documents from the same real articles. We can see that only the similarity score between the 7th and 9th documents is very high, and some scores are even lower than 0.5. For example, the similarity score between the 4th and 6th documents is as low as 0.13. 

In [8]:
## LSA, 3 topics
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=3) # initialize an LSI transformation
corpus_lsi = lsi[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi
lsi.print_topics()
#lsi.save('/tmp/model.lsi') # same for tfidf, lda, ...
#lsi = models.LsiModel.load('/tmp/model.lsi')

[(0,
  '0.420*"motors" + 0.420*"general" + 0.367*"tesla" + 0.294*"new" + 0.248*"many" + 0.221*"really" + 0.203*"cloud" + 0.187*"digitalocean" + 0.187*"smaller" + 0.186*"amazon"'),
 (1,
  '-0.466*"tests" + -0.466*"nuclear" + -0.466*"korea" + -0.455*"north" + 0.178*"general" + 0.178*"motors" + -0.172*"says" + 0.163*"tesla" + 0.109*"many" + 0.088*"new"'),
 (2,
  '0.360*"microsoft" + 0.360*"startup" + 0.360*"amazon" + 0.347*"cloud" + -0.298*"tesla" + 0.275*"smaller" + 0.275*"digitalocean" + -0.267*"motors" + -0.267*"general" + -0.195*"many"')]

The result shows that 
* "moters", "gerneral", and "tesla" are the main words for topic 0; 
* "tests", "nuclear", "korea", and "north" are the main words for topic 1; 
* "microsoft", "startup", "amazon", and "cloud" are the main words for topic 2. 

This result correspond to the real articles, so this result is very good.

In [9]:
for doc in corpus_lsi: # both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
     print(doc)

[(0, 0.63136333419162716), (1, 0.2418747215151914), (2, -0.21639515843351054)]
[(0, 0.76473285042256411), (1, 0.30785533014678573), (2, -0.46387836429245899)]
[(0, 0.64723103904767609), (1, 0.28226326197469598), (2, -0.47996544169683691)]
[(0, 0.37511819468735952), (1, 0.020548417765392658), (2, 0.70922771947860541)]
[(0, 0.53749758430847616), (1, 0.043592084259660165), (2, 0.76779476684524728)]
[(0, 0.43247406680499478), (1, -0.07070693129592541), (2, 0.46257622856120884)]
[(0, 0.25853906923354486), (1, -0.92063230557872544), (2, -0.19302674148562632)]
[(0, 0.23399311108509799), (1, -0.40748392597694622), (2, 0.098469374364577519)]
[(0, 0.25853906923354486), (1, -0.92063230557872544), (2, -0.19302674148562632)]


The result shows that 
* the first three documents belong to topic 0 ("moters", "gerneral", and "tesla) as they all have the highest scores in topic 0 (0.63, 0.76, 0.64);
* the middle three documents belong to topic 2 ("microsoft", "startup", "amazon", and "cloud") as they all have the highest scores in topic 2 (0.70, 0.76, 0.46);
* the last three documents belog to topic 1 ("tests", "nuclear", "korea", and "north")as they all have the highest (absolute) scores in topic 1 (-0.92, -0.40, -0.92).
* however, for the 6th document, the scores on topic 0 and topic 2 are almost equal (0.43 and 0.46), which are not correspond to the truth.

In gernal, the results correspond to the real topics as these documents come from three different real articles.

In [10]:
## LDA, 3 topics
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=3)
corpus_lda = lda[corpus] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi
lda.print_topics()

[(0,
  '0.149*"cloud" + 0.105*"digitalocean" + 0.105*"microsoft" + 0.105*"smaller" + 0.104*"startup" + 0.104*"amazon" + 0.062*"says" + 0.061*"really" + 0.060*"new" + 0.020*"north"'),
 (1,
  '0.205*"motors" + 0.202*"general" + 0.156*"tesla" + 0.110*"many" + 0.064*"really" + 0.063*"new" + 0.020*"north" + 0.019*"says" + 0.019*"tests" + 0.019*"korea"'),
 (2,
  '0.201*"north" + 0.145*"nuclear" + 0.143*"tests" + 0.142*"korea" + 0.081*"says" + 0.026*"general" + 0.025*"tesla" + 0.025*"cloud" + 0.024*"amazon" + 0.024*"motors"')]

The result shows that 
* "cloud", "digitalocean", and "microsoft" are the main words for topic 0; 
* "motors", "general", and "tesla" are the main words for topic 1; 
* "north", "nuclear", "test", and "korea" are the main words for topic 2. 

This result correspond to the real articles, so this result is very good.

In [11]:
for doc in corpus_lda: # both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
     print(doc)

[(0, 0.087835996929488125), (1, 0.82822853141194797), (2, 0.08393547165856384)]
[(0, 0.042608028523989247), (1, 0.91550625314149969), (2, 0.041885718334511039)]
[(0, 0.055740848923619774), (1, 0.88844974656088427), (2, 0.05580940451549584)]
[(0, 0.86595510378700524), (1, 0.06695528745825173), (2, 0.067089608754742985)]
[(0, 0.91527939994434671), (1, 0.042745536502851553), (2, 0.041975063552801811)]
[(0, 0.88419303837528773), (1, 0.057630925626454249), (2, 0.058176035998258033)]
[(0, 0.066916255049984802), (1, 0.066919645162085706), (2, 0.86616409978792963)]
[(0, 0.11898735642809571), (1, 0.11188070301041968), (2, 0.76913194056148459)]
[(0, 0.066916270540941777), (1, 0.066919614888273402), (2, 0.8661641145707849)]


The result shows that 
* the first three documents belong to topic 1 ("moters", "gerneral", and "tesla) as they all have the highest scores in topic 1 (0.82, 0.91, 0.88);
* the middle three documents belong to topic 0 ("cloud", "digitalocean", and "microsoft") as they all have the highest scores in topic 0 (0.86, 0.91, 0.88);
* the last three documents belog to topic 2 ("tests", "nuclear", "korea", and "north")as they all have the highest (absolute) scores in topic 2 (0.86, 0.76, 0.86).

This result corresponds to the real topics as these documents come from three different real articles.

#### Conclusions: From the examples above, we can see that the method of LDA is the best, the method of LSA is in the middle, and the method of TF-IDF is the worst.